# Parte 4: Aprendizaje federado con promedios de modelos

Hasta ahora, entrenamos un modelo utilizando una versión muy simple de Aprendizaje Federado. Esto requería que cada propietario de datos confiara en el propietario del modelo para poder ver sus gradientes.

Ahora, se mostrará cómo usar las herramientas avanzadas para permitir que un "trabajador seguro" en el que se confie agregue los pesos antes de que el modelo resultante final se devuelva al propietario del modelo (nosotros).

De esta manera, solo el trabajador seguro puede ver qué pesos provienen de quién. Podríamos decir qué partes del modelo cambiaron, pero NO sabemos qué trabajador (bob o alice) hizo qué cambio, lo que crea una capa de privacidad.


In [1]:
import syft as sy
import copy
hook = sy.TorchHook()
from torch import nn, optim

# Paso 1: Crear los propietarios de los datos.

Primero, vamos a crear dos propietarios de datos (Bob y Alice) cada uno con una pequeña cantidad de datos. También vamos a inicializar una máquina segura llamada "secure_worker". En la práctica, esto podría ser un hardware seguro (como el SGX de Intel) o simplemente un intermediario confiable.

In [2]:
# create a couple workers

bob = sy.VirtualWorker(id="bob")
alice = sy.VirtualWorker(id="alice")
secure_worker = sy.VirtualWorker(id="secure_worker")

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

# A Toy Dataset
data = sy.Var(sy.FloatTensor([[0,0],[0,1],[1,0],[1,1]]))
target = sy.Var(sy.FloatTensor([[0],[0],[1],[1]]))

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

# Paso 2: Crear nuestro modelo

Para este ejemplo, vamos a entrenar con un modelo lineal simple. Podemos inicializarlo normalmente usando el constructor nn.Linear de PyTorch.

In [3]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

# Paso 3: enviar una copia del modelo a Alice y Bob

Luego, debemos enviar una copia del modelo actual a Alice y Bob para que puedan realizar los pasos de aprendizaje en sus propios conjuntos de datos.

In [4]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

# Paso 4: Entrenar los modelos de Bob y Alice (en paralelo)

Como es convencional con Aprendizaje Federado a través de esta técnica de un tercero confiable, cada propietario de datos primero entrena su modelo para varias iteraciones a nivel local antes de que los modelos se promedien juntos.

In [5]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data[0]

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data[0]
    alices_loss

# Paso 5: Enviar ambos modelos actualizados a un trabajador seguro

Ahora que cada propietario de datos tiene un modelo parcialmente entrenado, es hora de promediarlos de manera segura. Esto lo logramos indicando a Alice y Bob que envíen su modelo al servidor seguro (confiable).


In [6]:
alices_model.move(secure_worker)
bobs_model.move(secure_worker)

Linear(in_features=2, out_features=1, bias=True)

# Paso 6: Promedio de los modelos

Finalmente, el último paso para esta etapa de entrenamiento es promediar los modelos entrenados de Bob y Alice y luego usarlos para establecer los valores de nuestro "modelo" global.

In [7]:
model.weight.data.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
model.bias.data.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
""

''

# Repetir proceso



In [8]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):
    
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

    for wi in range(worker_iters):

        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target)**2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data[0]

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target)**2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data[0]
    
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)
    
    model.weight.data.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.data.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:0.014401242136955261 Alice:0.021205957978963852
Bob:0.005423392169177532 Alice:0.011188073083758354
Bob:0.0018657767213881016 Alice:0.0051170047372579575
Bob:0.0005638035945594311 Alice:0.0022938279435038567
Bob:0.00013662339188158512 Alice:0.001032400643453002
Bob:2.3314680220209993e-05 Alice:0.00046972226118668914
Bob:9.560068065184169e-06 Alice:0.00021691148867830634
Bob:1.931003134814091e-05 Alice:0.00010205955186393112
Bob:2.9013930543442257e-05 Alice:4.9136768211610615e-05
Bob:3.363085852470249e-05 Alice:2.4316530470969155e-05


Por último, nos aseguramos de que nuestro modelo resultante haya aprendido correctamente, por lo que lo evaluamos en un conjunto de datos de prueba.

In [9]:
preds = model(data)
loss = ((preds - target) ** 2).sum()

In [10]:
print(preds)
print(target)
print(loss.data[0])

Variable containing:
 0.0233
 0.0231
 0.9699
 0.9697
[syft.core.frameworks.torch.tensor.FloatTensor of size 4x1]

Variable containing:
 0
 0
 1
 1
[syft.core.frameworks.torch.tensor.FloatTensor of size 4x1]

0.0028978544287383556


In this toy example, the averaged model is underfitting relative to a plaintext model trained locally would behave, however we were able to train it without exposing each worker's training data.  We were also able to aggregate the updated models from each worker on a trusted aggregator to prevent data leakage to the model owner.

In a future tutorial, we'll aim to do our trusted aggregation directly with the gradients, so that we can update the model with better gradient estimates and arrive at a stronger model.